In [1]:
using Distributions
using Roots
using StatsFuns
using StatsPlots
default(fmt=:png)

safemul(x, y) = x == 0 ? x : isinf(x) ? oftype(x, Inf) : x*y
safediv(x, y) = x == 0 ? x : isinf(y) ? zero(y) : x/y

oddsratiohat(a, b, c, d) = safediv(a*d, b*c)

function delta(a, b, c, d; ω=1)
    A, B, C = 1-ω, a+d+ω*(b+c), a*d-ω*b*c
    isinf(ω) ? oftype(ω, -min(b, c)) : safediv(2C, B + √(B^2 - 4A*C))
end

# correction = 0.5 は連続性補正を与える.
function _chisqstat_or(a, b, c, d, δ; correction=0.0)
    ã, b̃, c̃, d̃ = a-δ, b+δ, c+δ, d-δ
    safemul(max(0, abs(δ)-correction)^2, 1/ã + 1/b̃ + 1/c̃ + 1/d̃)
end

function chisqstat_or(a, b, c, d; ω=1, correction=0.0)
    δ = delta(a, b, c, d; ω)
    _chisqstat_or(a, b, c, d, δ; correction)
end

function pvalue_or_pearson_chisq(a, b, c, d; ω=1, correction=0.0)
    χ² = chisqstat_or(a, b, c, d; ω, correction)
    ccdf(Chisq(1), χ²)
end

function confint_or_pearson_chisq(a, b, c, d; α=0.05, correction=0.0)
    (a+b==0 || c+d==0 || a+c==0 || b+d==0) && return [0, Inf]
    f(logω) = logit(pvalue_or_pearson_chisq(a, b, c, d; ω=exp(logω), correction)) - logit(α)
    ps = if a == 0 || d == 0
        [0, exp(find_zero(f, 0.0))]
    elseif b == 0 || c == 0
        [exp(find_zero(f, 0.0)), Inf]
    else
        ORhat = oddsratiohat(a, b, c, d)
        ω_L, ω_U = ORhat/2, 2ORhat
        [exp(find_zero(f, log(ω_L))), exp(find_zero(f, log(ω_U)))]
    end
end

confint_or_pearson_chisq (generic function with 1 method)

<img src="IMG_8592.png">

In [2]:
A = [
    14 41981-14
     1 18327-1
]
a, b, c, d = A'

@show A
@show oddsratiohat(a, b, c, d)
@show confint_or_pearson_chisq(a, b, c, d)
@show pvalue_or_pearson_chisq(a, b, c, d)
;

A = [14 41967; 1 18326]
oddsratiohat(a, b, c, d) = 6.11347010746539
confint_or_pearson_chisq(a, b, c, d) = [1.0277686835274809, 36.36317619246635]
pvalue_or_pearson_chisq(a, b, c, d) = 0.045734383713917776


In [3]:
A = [
    14 41981-14
     2 18327-2
]
a, b, c, d = A'

@show A
@show oddsratiohat(a, b, c, d)
@show confint_or_pearson_chisq(a, b, c, d)
@show pvalue_or_pearson_chisq(a, b, c, d)
;

A = [14 41967; 2 18325]
oddsratiohat(a, b, c, d) = 3.056568256010675
confint_or_pearson_chisq(a, b, c, d) = [0.7754073277864398, 12.048222656197431]
pvalue_or_pearson_chisq(a, b, c, d) = 0.11971064589398882


In [4]:
A = [
    11 17969-11
     1 18327-1
]
a, b, c, d = A'

@show A
@show oddsratiohat(a, b, c, d)
@show confint_or_pearson_chisq(a, b, c, d)
@show pvalue_or_pearson_chisq(a, b, c, d)
;

A = [11 17958; 1 18326]
oddsratiohat(a, b, c, d) = 11.225414856888294
confint_or_pearson_chisq(a, b, c, d) = [1.8629687749595616, 67.6349343557417]
pvalue_or_pearson_chisq(a, b, c, d) = 0.0034830100353045444


In [5]:
A = [
    11 17969-11
     2 18327-2
]
a, b, c, d = A'

@show A
@show oddsratiohat(a, b, c, d)
@show confint_or_pearson_chisq(a, b, c, d)
@show pvalue_or_pearson_chisq(a, b, c, d)
;

A = [11 17958; 2 18325]
oddsratiohat(a, b, c, d) = 5.612401158258158
confint_or_pearson_chisq(a, b, c, d) = [1.3954333945865385, 22.571783961847302]
pvalue_or_pearson_chisq(a, b, c, d) = 0.011332010573363192


In [9]:
A = [
    11 17969-11
     3 18327-3
]
a, b, c, d = A'

@show A
@show oddsratiohat(a, b, c, d)
@show confint_or_pearson_chisq(a, b, c, d)#correction=0.5)
@show pvalue_or_pearson_chisq(a, b, c, d)#correction=0.5)
;

A = [11 17958; 3 18324]
oddsratiohat(a, b, c, d) = 3.7413965920481123
confint_or_pearson_chisq(a, b, c, d) = [1.1235405836250592, 12.458362906548109]
pvalue_or_pearson_chisq(a, b, c, d) = 0.029591125356557564


In [10]:
A = [
    11 17969-11
     3 18327-3
]
a, b, c, d = A'

@show A
@show oddsratiohat(a, b, c, d)
@show confint_or_pearson_chisq(a, b, c, d; correction=0.5)
@show pvalue_or_pearson_chisq(a, b, c, d; correction=0.5)
;

A = [11 17958; 3 18324]
oddsratiohat(a, b, c, d) = 3.7413965920481123
confint_or_pearson_chisq(a, b, c, d; correction = 0.5) = [0.9729294587795231, 16.864414230409725]
pvalue_or_pearson_chisq(a, b, c, d; correction = 0.5) = 0.05636604270757734
